In [375]:
import torch 
from torch import nn
from torch import FloatTensor
from torch.autograd import Variable
from torch import optim

from numpy.linalg import matrix_rank
from sklearn.datasets import make_spd_matrix
import os
from networkx import nx
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [47]:
temp = torch.nn.Linear(5, 3)
b = Variable(torch.rand((5, 5)))
a = Variable(torch.rand((2, 5, 5)))

In [50]:
b

Variable containing:
 0.5040  0.6785  0.2569  0.6513  0.8617
 0.4109  0.0840  0.2289  0.8198  0.9596
 0.9519  0.0509  0.2460  0.5015  0.9868
 0.0956  0.9169  0.4598  0.3063  0.6225
 0.6994  0.1210  0.9268  0.7742  0.5865
[torch.FloatTensor of size 5x5]

In [39]:
a[0]

Variable containing:
 0.3181  0.3285  0.7137  0.8696  0.6389
 0.2745  0.7231  0.4505  0.7706  0.4151
 0.1746  0.9885  0.1559  0.9463  0.3336
 0.2589  0.9943  0.8707  0.6326  0.0786
 0.3552  0.6404  0.1191  0.3398  0.8872
[torch.FloatTensor of size 5x5]

In [209]:
class Linear2D(nn.Module):
    
    def __init__(self, in_features, out_features):
        super(Linear2D, self).__init__()
        self.W = nn.Parameter(torch.rand((out_features, in_features)))
        
    def forward(self, X):
        XW = self._matmul(X, self.W.transpose(0 ,1))
        WXW = self._matmul(XW, self.W, kind='left')
        return WXW
        
    def _matmul(self, X, Y, kind='right'):
        results = [] 
        for i in range(X.size(0)):
            if kind == 'right':
                result = torch.mm(X[i], Y)
            elif kind == 'left':
                result = torch.mm(Y, X[i])
            results.append(result.unsqueeze(0))
        return torch.cat(results, 0)
    
    def check_rank(self):
        return min(self.W.data.size()) == matrix_rank(self.W.data.numpy())

In [210]:
class SymmetricallyCleanLayer(nn.Module):
    def __init__(self):
        super(SymmetricallyCleanLayer, self).__init__()
        self.relu = nn.ReLU()
        
    def forward(self, X):
        return self.relu(X) 

In [211]:
class NonLinearityBlock(nn.Module):
    def __init__(self, activation):
        super(NonLinearityBlock, self).__init__()
        self.activation = activation
        
    def forward(self, X):
        return self.activation(X)

In [218]:
class ShrinkBlock(nn.Module):
    def __init__(self, features):
        super(ShrinkBlock, self).__init__()
        self.weights_matrix = nn.Parameter(torch.zeros((features, features)).normal_())
        self.weights_matrix_square = torch.mm(self.weights_matrix.transpose(0, 1), self.weights_matrix)
        self.inverse_eye = Variable(torch.ones((features, features)) - torch.eye(features))
    
    def forward(self, X):
        return X - self.weights_matrix_square * self.inverse_eye

In [231]:
class MatrixEncoder(nn.Module):
    def __init__(self, n_features):
        super(MatrixEncoder, self).__init__()
        self.n_features = n_features
        
        self.encoder = nn.Sequential(
            Linear2D(n_features, 5),
            ShrinkBlock(5),
            SymmetricallyCleanLayer(),
            Linear2D(5, 3),
            ShrinkBlock(3)
        )
        
        self.decoder = nn.Sequential(
            Linear2D(3, 5),
            ShrinkBlock(5),
            SymmetricallyCleanLayer(),
            Linear2D(5, n_features),
            ShrinkBlock(n_features)
        )
        
    def forward(self, X):
        return self.decoder(self.encoder(X))

In [359]:
def laplacian_tensor(x):
    int_array = np.squeeze(np.asarray(nx.laplacian_matrix(x).todense()))
    return int_array.astype('float64')

In [245]:
from torch.nn.modules.loss import MSELoss

In [279]:
coder = MatrixEncoder(6)
matrix = Variable(FloatTensor(make_spd_matrix(6))).unsqueeze(0)
optimizer = optim.Adam(coder.parameters(), lr=0.01)
criterion = MSELoss()

for epoch in range(4000):
    optimizer.zero_grad()
    outputs = coder(matrix)
    loss = criterion(outputs, matrix)
    loss.backward(retain_graph=True)
    if epoch % 100 == 0:
        print(loss.data[0])
    optimizer.step()

85001.234375
164.30828857421875
85.11470031738281
52.89616012573242
36.56184387207031
27.230751037597656
21.408578872680664
17.52837562561035
14.807859420776367
12.82198715209961
11.323862075805664
10.197795867919922
9.350672721862793
8.643263816833496
8.04478931427002
7.532347679138184
7.088569164276123
6.700085639953613
6.356471538543701
6.04948616027832
5.772524356842041
5.520229816436768
5.28825044631958
5.075875759124756
4.876154899597168
4.686581611633301
4.505087375640869
4.329866409301758
4.159339427947998
3.992133140563965
3.8274006843566895
3.6646995544433594
3.50199556350708
3.338881254196167
3.1752147674560547
3.01114559173584
2.8471343517303467
2.6839442253112793
2.5226051807403564
2.3643441200256348


In [362]:
graphs = []
for graph in os.listdir('./test_graphs/')[1:]:
    graphs.append(nx.read_graphml('./test_graphs/' + graph))

In [369]:
graphs[0].remove_nodes_from(list(graphs[0].nodes)[10:])

In [373]:
graphs = []
for graph in os.listdir('./test_graphs/')[1:]:
    graphs.append(nx.read_graphml('./test_graphs/' + graph))

In [381]:
def read_test_graphs():
    graphs = []
    for graph in os.listdir('./test_graphs/')[1:]:
        graphs.append(nx.read_graphml('./test_graphs/' + graph))
    return graphs

def build_dataset(graphs, nodes_number=6):
    laplacian_matrixes
    for graph in graphs:
        graph.remove_nodes_from(list(graph.nodes)[nodes_number:])
    
    return graphs

In [383]:
t = build_dataset(read_test_graphs())

In [401]:
def laplacian_tensor(x):
    return FloatTensor(np.squeeze(np.asarray(nx.laplacian_matrix(x).todense().astype('float64'))))

In [402]:
laplacian_tensor(t[0])


 0  0  0  0  0  0
 0  2 -1  0  0 -1
 0 -1  1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0 -1  0  0  0  1
[torch.FloatTensor of size 6x6]